In [2]:
pip install selenium


The following command must be run outside of the IPython shell:

    $ pip install selenium

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [3]:
from bs4 import BeautifulSoup

In [4]:

from bs4 import BeautifulSoup
import requests
import csv
import re
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from openpyxl import Workbook

In [5]:
#生成需要爬取的日期
# 傻葉子，如果你不是爬取每一天的数据，你可以改变这个函数
import datetime
def create_assist_date(datestart = None,dateend = None):
# 创建日期辅助表
    if datestart is None:
        datestart = '2016-01-01'
    if dateend is None:
        dateend = '2020-3-1'    #这个参数是我的研究范围决定的，你记得按你自己的改一下
    # 转为日期格式
    datestart=datetime.datetime.strptime(datestart,'%Y-%m-%d')
    dateend=datetime.datetime.strptime(dateend,'%Y-%m-%d')
    date_list = []
    date_list.append(datestart.strftime('%Y-%m-%d'))
    while datestart<dateend:
        # 日期叠加一天
        datestart+=datetime.timedelta(days=+1)
        # 日期转字符串存入列表
        date_list.append(datestart.strftime('%Y-%m-%d'))
    return date_list

In [6]:
#上述方法的一个实例化
date_crawler_2018=create_assist_date("2020-01-24","2020-02-03")
date_crawler_2018

['2020-01-24',
 '2020-01-25',
 '2020-01-26',
 '2020-01-27',
 '2020-01-28',
 '2020-01-29',
 '2020-01-30',
 '2020-01-31',
 '2020-02-01',
 '2020-02-02',
 '2020-02-03']

In [7]:
def get_url_list(data_crawler):
    k = len(data_crawler)
    url0 = []
    for i in range(0, k):
        # 我这里关键词是A股，你可以改成你想要的关键词
        str1 = 'https://s.weibo.com/weibo?q=新型肺炎&xsort=hot&suball=1&timescope=custom:'
        str2 = data_crawler[i]
        str3 = data_crawler[i]
        str4 = '&Refer=g'
        str5 = 'page='
        url0.append(str1+str2+':'+str3+str4+str5)
    return url0

In [18]:
# 这个东西是用来获取连续访问的cookie的
def get_weibo_cookies():
    url1 = ''
    
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument('--headless')
    chromeOptions.add_argument('--no-sandbox')
    chromeOptions.add_experimental_option('useAutomationExtension', False)
    capabilities = {'chromeOptions': {'useAutomationExtension': False, 'args': ['--disable-extensions']}}
    #driver = webdriver.Chrome(executable_path='C://Users//mi//Desktop//chromedriver.zip//chromedriver.exe',options=chromeOptions,desired_capabilities=capabilities)
    driver = webdriver.Chrome(executable_path='C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe',options=chromeOptions,desired_capabilities=capabilities)
    #driver = webdriver.Chrome(executable_path='C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe')

    driver.get(url1)
    driver.implicitly_wait(5)
    driver.find_element_by_xpath('//*[@id="username"]').clear()
    driver.find_element_by_xpath('//*[@id="username"]').send_keys('xx')# 你的账号
    driver.find_element_by_xpath('//*[@id="password"]').clear()
    driver.find_element_by_xpath('//*[@id="password"]').send_keys('xx')#你的密码
    driver.find_element_by_xpath('//*[@id="password"]').send_keys(Keys.ENTER)
    time.sleep(10)

# 傻葉子记得把下面的参数改一改：
# q是你要搜索的关键词，xsort代表热门，timescope是时间范围，比如我这里时间参数是1月1号8点到17点
    driver.get('https://s.weibo.com/weibo?q=A%E8%82%A1&scope=ori&suball=1&timescope=custom:2020-01-22-0:2020-02-03-23&Refer=SWeibo_box')

    cookies = driver.get_cookies()
    ret = ''
    for cookie in cookies:
        cookie_name = cookie['name']
        print(cookie_name)
        cookie_value = cookie['value']
        print(cookie_value)
        ret = ret+cookie_name+'='+cookie_value+';'
    return ret

In [30]:
pip install Soup

Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-cp37-none-any.whl size=1278 sha256=e55e14588738995c78b566a3502996c68946009871440373491e0908a52426ca
  Stored in directory: C:\Users\mi\AppData\Local\pip\Cache\wheels\a0\b0\b2\4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#爬取某年的评论数据

url_not=[] #存储爬取失败的,提高鲁棒
urls = get_url_list(date_crawler_2018)
#下面这个header你可能要改一下，我不知道你的User-Agent版本。
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363',
          'cookie': get_weibo_cookies()
          }  # 请求头，模拟浏览器登陆

page = list(range(1,51,1))#新浪高级搜索只能搜索到50页一次

#下面可能参数我不知道新浪有没有变化，傻葉子你先看看符不符合你的需求吧

namelist = []
textlist = []
favouritelist = []
forwardlist=[]
commentlist=[]
likelist=[]
datelist = []

for url in urls:
    n=urls.index(url)
    for i in page:  # 循环访问新浪的网页
        #sessions = requests.session()
        url_temp = url+str(i)
        response = requests.get(url_temp, headers=headers)
        if response.status_code != 200:
            url_not.append(url_temp)
            print("try: " + url_temp + ' later')
        soup = BeautifulSoup(response.text) 
        if list(soup.find_all('div',class_='m-error')):
            break
        else:
            # 提取名字
            names = soup.find_all('a', class_='name')
            for name in names:
                namelist.append(name.get_text())

            # 提取评论信息
            texts = soup.find_all('p', class_='txt')  # , attrs={'node-type': 'feed_list_content_full'})
            for text in texts:
                text_test = texts.index(text) + 1
                if texts.index(text) == len(texts) - 1:
                    textlist.append(text.get_text())
                    break
                elif texts[text_test].attrs['node-type'] == 'feed_list_content_full':
                    continue
                textlist.append(text.get_text())

            # 提取收藏数量
            favourite = soup.find_all('a',attrs={"action-type":"feed_list_favorite"})
            for f in favourite:
                favouritelist.append(f.get_text())
                
            # 提取转发数量
            forward = soup.find_all('a',attrs={"action-type":"feed_list_forward"})
            for f in forward:
                forwardlist.append(f.get_text())
                
            # 提取评论数量
            comment = soup.find_all('a',attrs={"action-type":"feed_list_comment"})
            for f in comment:
                commentlist.append(f.get_text())
                
            # 提取点赞数量
            like = soup.find_all('a',attrs={"action-type":"feed_list_like"})
            for f in like:
                likelist.append(f.get_text())

            # 提取时间,这里可以后续做进一步正则表达式处理
            dates = soup.find_all('p', class_='from')
            for date in dates:
                datelist.append(date.get_text())
            
        time.sleep(random.random())
        print(n,i) # 第n个时间段的url下搜索结果的第i页爬取完毕


In [30]:
from openpyxl import Workbook
wb = Workbook()    #创建文件对象
ws = wb.active     #获取第一个sheet
Num = len(namelist)
content_name=['name','text','favourite','forward','comment','like','date']
ws.append(content_name)
for i in range(0,Num):
    content=[]
    content.append(namelist[i])
    content.append(textlist[i])
    content.append(favouritelist[i])
    content.append(forwardlist[i])  
    content.append(commentlist[i])
    content.append(likelist[i])
    content.append(datelist[i])
    ws.append(content)
#wb.save("C://Users//mi//Desktop//data.xlsx")
wb.save("./data.xlsx")

                    

In [31]:
namelist

['吃瓜群众CJ',
 '新京报我们视频',
 '罗超super',
 '人民日报',
 '人民网',
 '央视新闻',
 '史上第一最牛逼',
 '头条新闻',
 '人民日报',
 '人民日报',
 '中国日报',
 '梨视频',
 '新京报我们视频',
 '腾讯公司',
 '央视新闻',
 '头条新闻',
 '胡锡进',
 '一手Video',
 '新浪视频',
 '吃瓜群众CJ',
 '新京报我们视频',
 '罗超super',
 '人民日报',
 '人民网',
 '央视新闻',
 '史上第一最牛逼',
 '头条新闻',
 '人民日报',
 '人民日报',
 '中国日报',
 '梨视频',
 '新京报我们视频',
 '腾讯公司',
 '央视新闻',
 '头条新闻',
 '胡锡进',
 '一手Video',
 '新浪视频',
 '吃瓜群众CJ',
 '新京报我们视频',
 '罗超super',
 '人民日报',
 '人民网',
 '央视新闻',
 '史上第一最牛逼',
 '头条新闻',
 '人民日报',
 '人民日报',
 '中国日报',
 '梨视频',
 '新京报我们视频',
 '腾讯公司',
 '央视新闻',
 '头条新闻',
 '胡锡进',
 '一手Video',
 '新浪视频',
 '吃瓜群众CJ',
 '新京报我们视频',
 '罗超super',
 '人民日报',
 '人民网',
 '央视新闻',
 '史上第一最牛逼',
 '头条新闻',
 '人民日报',
 '人民日报',
 '中国日报',
 '梨视频',
 '新京报我们视频',
 '腾讯公司',
 '央视新闻',
 '头条新闻',
 '胡锡进',
 '一手Video',
 '新浪视频',
 '吃瓜群众CJ',
 '新京报我们视频',
 '罗超super',
 '人民日报',
 '人民网',
 '央视新闻',
 '史上第一最牛逼',
 '头条新闻',
 '人民日报',
 '人民日报',
 '中国日报',
 '梨视频',
 '新京报我们视频',
 '腾讯公司',
 '央视新闻',
 '头条新闻',
 '胡锡进',
 '一手Video',
 '新浪视频',
 '吃瓜群众CJ',
 '新京报我们视频',
 '罗超super',
 '人民日报',
 '人民网',
 '央视新闻',
 '史上第一最牛逼',
 '

In [32]:
textlist

['\n                    李佳琦和薇娅都给武汉捐了医疗物资。薇娅捐了100万给武汉慈善总会新型肺炎专项基金，李佳琦在这会儿的直播里，也把基金的捐赠公式公布在画面最显眼的地方。刚刚才在春晚舞台上比划完灯笼的老舅，宣布要把《野狼disco》的所有版权收入捐给在武汉的医务人员家属。真好，每个人都尽自己所能地给予着那些忙碌的伟大身影们，坚定的支持和爱。 收起全文d\n',
 '\n#武汉小汤山年夜饭#武汉“小汤山”医院开建，施工工人路边站立吃年夜饭。#聚焦新型肺炎# L我们视频拍者的秒拍视频 #武汉版小汤山工人年夜饭# \u200b                ',
 '\n                    接上条，武汉急需医用物资。这是无数一线医务奋战者的实时真实状态：物资匮乏，医护人员的付出远超于对他们自身的保护，这就是目前有些让人绝望的状态。但是，我们仍抱希望，哪怕你的一次转发都是对武汉人的助力！快递，可顺丰邮政均可到！#全国确诊新型肺炎病例# \u200b                ',
 '\n                    【#青海发现首例新型肺炎疑似病例# 患者27岁】患者陈某某，27岁，男性，在湖北省武汉市工作，2020年1月21日自武汉来宁探亲，现暂居我省西宁市。1月23日下午自行前往西宁市指定定点医疗机构就诊，经筛查即被医学隔离观察。患者标本经省疾病预防控制中心实验室检测，均呈新型冠状病毒核酸阳性，判定为新型冠状病毒感染的肺炎疑似病例。1月24日，患者标本已及时送至中国疾病预防控制中心进行复核检测。现患者病情稳定，患者密切接触者已开展追踪调查和医学观察。 收起全文d\n',
 '\n                    【#四川启动一级响应#】1月24日，四川省委副书记、省长尹力主持召开省应对新型冠状病毒感染的肺炎疫情联防联控机制领导小组第三次全体会议，研判当前疫情发展态势，进一步加强突发公共卫生事件应急机制，完善防控策略和措施。会议建议，根据当前全省防控工作需要，依法依规启动突发公共卫生事件一级应急响应。@四川发布 收起全文d\n',
 '\n                    【好消息！#浙江首例确诊新型肺炎患者痊愈出院#】今天14时，浙江温州市第六人民医院，温州首例确诊新型肺炎患者杨先生痊愈出院。据了

In [33]:
favouritelist

['收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',
 '收藏',

In [34]:
forwardlist

[' 转发 6037',
 ' 转发 63643',
 ' 转发 10944',
 ' 转发 5099',
 ' 转发 5422',
 ' 转发 3842',
 ' 转发 4330',
 ' 转发 2531',
 ' 转发 5007',
 ' 转发 1482',
 ' 转发 1162',
 ' 转发 481',
 ' 转发 570',
 ' 转发 1866',
 ' 转发 1088',
 ' 转发 296',
 ' 转发 2292',
 ' 转发 5395',
 ' 转发 638',
 ' 转发 6037',
 ' 转发 63643',
 ' 转发 10944',
 ' 转发 5099',
 ' 转发 5422',
 ' 转发 3842',
 ' 转发 4330',
 ' 转发 2531',
 ' 转发 5007',
 ' 转发 1482',
 ' 转发 1162',
 ' 转发 481',
 ' 转发 570',
 ' 转发 1866',
 ' 转发 1088',
 ' 转发 296',
 ' 转发 2292',
 ' 转发 5395',
 ' 转发 638',
 ' 转发 6037',
 ' 转发 63643',
 ' 转发 10944',
 ' 转发 5099',
 ' 转发 5422',
 ' 转发 3842',
 ' 转发 4330',
 ' 转发 2531',
 ' 转发 5007',
 ' 转发 1482',
 ' 转发 1162',
 ' 转发 481',
 ' 转发 570',
 ' 转发 1866',
 ' 转发 1088',
 ' 转发 296',
 ' 转发 2292',
 ' 转发 5395',
 ' 转发 638',
 ' 转发 6037',
 ' 转发 63643',
 ' 转发 10944',
 ' 转发 5099',
 ' 转发 5422',
 ' 转发 3842',
 ' 转发 4330',
 ' 转发 2531',
 ' 转发 5007',
 ' 转发 1482',
 ' 转发 1162',
 ' 转发 481',
 ' 转发 570',
 ' 转发 1866',
 ' 转发 1088',
 ' 转发 296',
 ' 转发 2292',
 ' 转发 5395',
 ' 转发 638',
 ' 转发 6037',
 ' 转发 6

In [35]:
datelist

['\n\n                        01月24日 21:11\n                        \n                                         \xa0来自 iPhone客户端 ',
 '\n\n                        01月24日 20:29\n                        \n',
 '\n\n                        01月24日 19:57\n                        \n                                         \xa0来自 HUAWEI P30 Pro ',
 '\n\n                        01月24日 17:47\n                        \n                                         \xa0来自 微博 weibo.com ',
 '\n\n                        01月24日 18:04\n                        \n                                         \xa0来自 专业版微博 ',
 '\n\n                        01月24日 19:41\n                        \n                                         \xa0来自 微博云剪 ',
 '\n\n                        01月24日 18:46\n                        \n',
 '\n\n                        01月24日 22:24\n                        \n                                         \xa0来自 微博 weibo.com ',
 '\n\n                        01月24日 21:24\n                      